<a href="https://colab.research.google.com/github/pandov/myitacademy-pandov/blob/master/FMDepthRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext tensorboard
! sh colab.sh setup
! dvc pull stages/AFLW2000.dvc
! unzip "data/external/*.zip"

In [ ]:
%%capture
import torch
from catalyst.dl.utils import set_global_seed, prepare_cudnn
from src.kcv import CrossValidation, Experiment, experiment_run
from src.callbacks import *
from src.features import AFLW2000
from src.models import FMDepthRNN
set_global_seed(3)
prepare_cudnn(deterministic=True)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
class Ex1(Experiment):
    def model(self): return FMDepthRNN(hidden_size=136)
    def criterion(self): return torch.nn.MSELoss()
    def optimizer(self, model): return torch.optim.Adam(model.parameters(), lr=2e-3)
    def scheduler(self, optimizer): return torch.optim.lr_scheduler.StepLR(optimizer, step_size=75, gamma=0.5)
class Ex2(Ex1):
    def optimizer(self, model): return torch.optim.Adam(model.parameters(), lr=1e-3)
class Ex3(Ex1):
    def model(self): return FMDepthRNN(hidden_size=204)
    def optimizer(self, model): return torch.optim.Adam(model.parameters(), lr=2e-3, weight_decay=1e-5)
class Ex4(Ex2):
    def model(self): return FMDepthRNN(hidden_size=204)
    def optimizer(self, model): return torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

In [ ]:
%tensorboard --logdir logs

In [ ]:
%%capture
experiments = [Ex1(), Ex2(), Ex3(), Ex4()]
callbacks = [AccuracyCallback()]
cross_val = CrossValidation(AFLW2000(), 4)
logdir = 'logs/FMDepthRNN'
experiment_run(experiments, callbacks, cross_val, logdir, device, 'accuracy01', 100, 250)

In [ ]:
! sh colab.sh save $logdir

In [ ]:
# ! rm -r logs/FMDepthRNN